In [7]:
from threading import ThreadError

import pandas as pd
import numpy as np
import os

Đầu tiên chúng ta đọc và kiểm tra dữ liệu từ file movies.csv

In [8]:
movies = pd.read_csv(os.path.join("data2", "movies.csv"), encoding="UTF-8")

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Nhìn có vẻ ổn nhưng ở một vài phim bắt đầu bằng The, như The Avengers (2012) sẽ có một lỗi nhỏ như sau

In [10]:
movies[movies['movieId'] == 89745]

,movieId,title,genres
17071,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX


Có thể thấy, Title của The Avengers bị đổi thành Avengers, nên chúng ta sẽ tạo một hàm để sửa nó, Có 2 loại title, một là có số năm còn lại là không có, nên chúng ta phải sửa cho từng loại riêng nhau

In [11]:
import re
def fix_the_title(title):
    pattern_with_year = r'^(.+), The \((\d{4})\)$'
    pattern_without_year = r'^(.+), The$'
    match_with_year = re.match(pattern_with_year, title)
    if match_with_year:
        movie_name = match_with_year.group(1).strip()
        year = match_with_year.group(2)
        return f"The {movie_name} ({year})"
    match_without_year = re.match(pattern_without_year, title)
    if match_without_year:
        movie_name = match_without_year.group(1).strip()
        return f"The {movie_name}"
    return title

Giờ chúng ta áp dụng nó vào dataframe đã tạo

In [12]:
movies['title'] = movies['title'].apply(fix_the_title)
movies[movies['movieId'] == 89745]

,movieId,title,genres
17071,89745,The Avengers (2012),Action|Adventure|Sci-Fi|IMAX


Thành công, tiếp theo chúng ta kiểm tra dữ liệu của file ratings.csv, file này chứa các cột về đánh giá phim của id người dùng. Cột timestamp để thể hiện thời điểm người dùng đánh giá, nó là số giây từ ngày 1 tháng 1 năm 1970 ở giờ UTC

In [13]:
ratings = pd.read_csv(os.path.join("data2", "ratings.csv"), encoding="UTF-8")
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


Chúng ta sẽ bỏ cột timestamp vì nó không cần thiết và merge với df movies

In [14]:
ratings.drop('timestamp', inplace=True, axis=1)
ratings.dropna(inplace=True)

Bởi vì ratings chứa dữ liệu là data đánh giá của từng người dùng nên chúng ta phải group by nó bằng movie id để có thể gộp với df movies. Đồng thời tạo cột avg_rating tính điểm đánh giá trung bình của từng phim. UserID sẽ được dùng để đếm số lượt đánh giá

In [15]:
ratings_summary = ratings.groupby('movieId').agg({'rating': 'mean', 'userId': 'count'}).rename(
    columns={'rating': 'avg_rating', 'userId': 'num_votes'})
ratings_summary.head()

,avg_rating,num_votes
movieId,,
1,3.897438,68997
2,3.275758,28904
3,3.139447,13134
4,2.845331,2806
5,3.059602,13154


In [16]:
movies = movies.merge(ratings_summary, on='movieId', how='left')

In [17]:
movies.head()

,movieId,title,genres,avg_rating,num_votes
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.897438,68997.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.275758,28904.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.139447,13134.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.845331,2806.0
4,5,Father of the Bride Part II (1995),Comedy,3.059602,13154.0


Tiếp theo là file tag, chứa các thông tin về các đặc điểm của phim do người dùng tự đánh giá, không giống phần genres vì người dùng tự tạo nên có thể có các thông tin rõ ràng hơn

In [18]:
tags = pd.read_csv(os.path.join("data2", "tags.csv"), encoding="UTF-8")
tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


In [19]:
tags.drop('timestamp', inplace=True, axis=1)

In [20]:
tags.head()

,userId,movieId,tag
0,22,26479,Kevin Kline
1,22,79592,misogyny
2,22,247150,acrophobia
3,34,2174,music
4,34,2174,weird


Có một số tags được người dùng viết bằng ngôn ngữ khác như sau

In [21]:
tags[(tags['userId'] == 3033) & (tags['movieId'] == 3)]

,userId,movieId,tag
16946,3033,3,comedinha de velhinhos engraÃƒÂ§ada


Có thể thấy lỗi font rõ ràng cho dù chúng ta đã dùng encoding = UTF-8 khi tạo df, nên chúng ta sẽ tạo hàm để sửa nó

In [22]:
import ftfy
def fix_encoding(text):
    if pd.isna(text):
        return ""
    fixed_text = ftfy.fix_text(str(text))
    fixed_text = re.sub(r'[^\x00-\x7F\u00C0-\u017F]', '', fixed_text)
    return fixed_text.strip()

Áp dụng nó vào df tags

In [23]:
tags['tag'] = tags['tag'].apply(fix_encoding)
tags[(tags['userId'] == 3033) & (tags['movieId'] == 3)]

,userId,movieId,tag
16946,3033,3,comedinha de velhinhos engraçada


Chúng ta cũng sẽ group by movieId để có thể merge lại, đồng thời gộp tất cả tag của một phim thành một hàng, có một vài tag có thể chỉ là số năm như các phim tài liệu nên chúng ta sẽ phải chuyển tất cả thành type string

In [24]:
tags['tag'] = tags['tag'].astype(str)
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: " ".join(x)).reset_index()

In [25]:
tags_grouped.head()

,movieId,tag
0,1,children Disney animation children Disney Disn...
1,2,Robin Williams fantasy Robin Williams time tra...
2,3,comedinha de velhinhos engraçada comedinha de ...
3,4,characters slurs based on novel or book chick ...
4,5,Fantasy pregnancy remake family Steve Martin s...


Việc một phim có thể có trùng các tag sẽ giúp chúng ta gợi ý phim hơp lý hơn vì nó sẽ hoạt động như một trọng số trong các model

In [26]:
movies = movies.merge(tags_grouped, on='movieId', how='left')
movies['tag'] = movies['tag'].fillna("") #Có thể có một phim nào đó chưa được đánh giá sẽ không có tag

In [27]:
movies.head()

,movieId,title,genres,avg_rating,num_votes,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.897438,68997.0,children Disney animation children Disney Disn...
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.275758,28904.0,Robin Williams fantasy Robin Williams time tra...
2,3,Grumpier Old Men (1995),Comedy|Romance,3.139447,13134.0,comedinha de velhinhos engraçada comedinha de ...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.845331,2806.0,characters slurs based on novel or book chick ...
4,5,Father of the Bride Part II (1995),Comedy,3.059602,13154.0,Fantasy pregnancy remake family Steve Martin s...


tạo df links, chứa các id của phim trên các web như imdb và tmdb, dùng để lấy thông tin poster của từng phim khi tạo UI

In [34]:
links = pd.read_csv(os.path.join('data2', "links.csv"))
movies = movies.merge(links[['movieId', 'tmdbId']], on='movieId', how='left')

Chúng ta sẽ sử dụng các tag đã gộp để tạo ma trận TF-IDF, giúp đo lường độ tương đồng giữa các phim dựa trên nội dung của tag. Có thể xem là vừa lọc theo nội dung và vừa theo collaborative filtering vì tag là do người dùng tạo.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
def create_tfidf_matrix(movies):
    print("Bắt đầu tạo ma trận TF-IDF...")
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Sử dụng unigram và bigram
    tfidf_matrix = vectorizer.fit_transform(movies['tag'])
    print("Hoàn tất tạo ma trận TF-IDF.")
    return vectorizer, tfidf_matrix

Chúng ta sẽ tạo ba hàm gợi ý phim:
1. **Lọc dựa trên nội dung**: Dựa trên độ tương đồng của tag.
2. **Lọc cộng tác**: Dựa trên sở thích của người dùng tương tự.
3. **Lọc kết hợp**: Kết hợp cả hai phương pháp trên.


## Hàm Gợi Ý Dựa trên Nội Dung
Sử dụng ma trận TF-IDF để tìm các phim có tag tương tự.

In [29]:
def find_similar_movies_content_based(movie_id, movies, tfidf_matrix):
    print("Tìm phim tương tự dựa trên tag...")
    movie_idx = movies[movies['movieId'] == movie_id].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[movie_idx], tfidf_matrix).flatten()
    similar_indices = cosine_sim.argsort()[-6:-1][::-1]  # Lấy top 5, bỏ phim gốc
    print("Hoàn tất tìm phim tương tự.")
    return movies.iloc[similar_indices][['title', 'tmdbId']]

### Hàm Gợi Ý Dựa trên Lọc Cộng Tác
Dựa trên đánh giá của người dùng có sở thích tương tự.

In [30]:
def find_similar_movies_collaborative(movie_id, ratings, movies):
    print(f"Tìm phim tương tự cho movie_id: {movie_id}...")
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .1]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values(by="score", ascending=False)
    rec_percentages = rec_percentages[rec_percentages.index != movie_id]

    print("Hoàn tất tìm phim tương tự.")
    return rec_percentages.head(5).merge(movies, left_index=True, right_on="movieId")[["title", "tmdbId"]]


## Hàm Gợi Ý Kết Hợp
Kết hợp cả lọc dựa trên nội dung và lọc cộng tác để tạo gợi ý cân bằng.

In [31]:
def find_hybrid_recommendations(movie_id, ratings, movies, tfidf_matrix):
    print(f"Tìm phim tương tự bằng lọc kết hợp cho movie_id: {movie_id}...")
    collab_recs = find_similar_movies_collaborative(movie_id, ratings, movies).reset_index(drop=True)
    content_recs = find_similar_movies_content_based(movie_id, movies, tfidf_matrix).reset_index(drop=True)
    hybrid_recs = []
    for i in range(5):
        if i % 2 == 0 and i // 2 < len(collab_recs):
            hybrid_recs.append(collab_recs.iloc[i // 2])
        elif i // 2 < len(content_recs):
            hybrid_recs.append(content_recs.iloc[i // 2])
    hybrid_recs_df = pd.DataFrame(hybrid_recs, columns=['title', 'tmdbId'])
    print("Hoàn tất tìm phim tương tự bằng lọc kết hợp.")
    return hybrid_recs_df

## Lấy Áp Phích Phim từ TMDb
Để tăng tính trực quan, chúng ta sẽ lấy áp phích phim từ API TMDb dựa trên tmdbId.


In [32]:
import requests

TMDB_API_KEY = "ab3e3f106356dbcb70df22107bb51b09"

def get_poster_url(tmdb_id):
    """
    Lấy URL áp phích phim từ TMDb.

    Args:
        tmdb_id (float): ID TMDb của phim

    Returns:
        str: URL áp phích hoặc None nếu không có
    """
    if pd.isna(tmdb_id):
        return None
    url = f"https://api.themoviedb.org/3/movie/{int(tmdb_id)}?api_key={TMDB_API_KEY}&language=en-US"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        poster_path = data.get('poster_path')
        if poster_path:
            return f"https://image.tmdb.org/t/p/w185{poster_path}"
    return None


## Test các hàm gợi ý

In [35]:
vectorizer, tfidf_matrix = create_tfidf_matrix(movies)
print("\nGợi ý dựa trên nội dung cho Toy Story:")
print(find_similar_movies_content_based(1, movies, tfidf_matrix))
print("\nGợi ý cộng tác cho Toy Story:")
print(find_similar_movies_collaborative(1, ratings, movies))
print("\nGợi ý kết hợp cho Toy Story:")
print(find_hybrid_recommendations(1, ratings, movies, tfidf_matrix))

Bắt đầu tạo ma trận TF-IDF...
Hoàn tất tạo ma trận TF-IDF.

Gợi ý dựa trên nội dung cho Toy Story:
Tìm phim tương tự dựa trên tag...
Hoàn tất tìm phim tương tự.
                       title   tmdbId
3021      Toy Story 2 (1999)    863.0
2264    Bug's Life, A (1998)   9487.0
14815     Toy Story 3 (2010)  10193.0
4781   Monsters, Inc. (2001)    585.0
6259     Finding Nemo (2003)     12.0

Gợi ý cộng tác cho Toy Story:
Tìm phim tương tự cho movie_id: 1...
Hoàn tất tìm phim tương tự.
                       title   tmdbId
3021      Toy Story 2 (1999)    863.0
2264    Bug's Life, A (1998)   9487.0
14815     Toy Story 3 (2010)  10193.0
4781   Monsters, Inc. (2001)    585.0
580           Aladdin (1992)    812.0

Gợi ý kết hợp cho Toy Story:
Tìm phim tương tự bằng lọc kết hợp cho movie_id: 1...
Tìm phim tương tự cho movie_id: 1...
Hoàn tất tìm phim tương tự.
Tìm phim tương tự dựa trên tag...
Hoàn tất tìm phim tương tự.
Hoàn tất tìm phim tương tự bằng lọc kết hợp.
                  title   tmdbI